# Read Overpass-API to Python Pandas Dataframe - Luzern

The [Overpass API](http://wiki.openstreetmap.org/wiki/Overpass_API) provides access to the data behind the Openstreetmaps Map Data.
The [Overpass-Turbo](http://overpass-turbo.eu/) is the easyiest way to test requests and get the correct code to ask the database. Example: Fountains with wikidata Q-Numbers as per https://github.com/water-fountains/import2wikidata/issues/15

In [1]:
#TODO eventually import most functions from ../../generics

In [2]:
from datetime import datetime as dt
dtFmt = "%y%m%d_%H%M%S"
print (dt.now().strftime(dtFmt))
import pandas as pd
import requests
import json
pd.set_option('display.max_columns', 200)

191125_073832


In [13]:
# Get yours at: http://boundingbox.klokantech.com/
nameOfRegion='Luzern'
# Luzern ll 47.0091,8.2380 ur 47.0853,8.3584
bbox = [8.2380,47.0091,8.3584,47.0853]

# Links unten
minLat = bbox[1]
minLon = bbox[0]

# Rechts oben
maxLat = bbox[3]
maxLon = bbox[2]

### Construct the Overpass Query String

Request from [Overpass-Turbo](http://overpass-turbo.eu/)

In [4]:
def makeOsmQuery(minLat, minLon, maxLat, maxLon):
    bbox_string = '(%s,%s,%s,%s)' % (minLat, minLon, maxLat, maxLon)
    
    compactOverpassQLstring = f'''
    
    [out:json][timeout:60];
    
    (
        node[man_made=drinking_fountain][wikidata]{bbox_string};
        node[amenity=drinking_water][wikidata]{bbox_string};
        node[amenity=fountain][wikidata]{bbox_string};
    );'''
    
    
    #node[man_made=water_tap]{bbox_string};node[natural=spring]{bbox_string};node[amenity=watering_place]{bbox_string};
    #node[amenity=water_point]{bbox_string};node[man_made=water_well]{bbox_string};
    compactOverpassQLstring += 'out ;'
    return compactOverpassQLstring

In [5]:
compactOverpassQLstring = makeOsmQuery(minLat, minLon, maxLat, maxLon)
compactOverpassQLstring

'\n    \n    [out:json][timeout:60];\n    \n    (\n        node[man_made=drinking_fountain][wikidata](47.0091,8.238,47.0853,8.3584);\n        node[amenity=drinking_water][wikidata](47.0091,8.238,47.0853,8.3584);\n        node[amenity=fountain][wikidata](47.0091,8.238,47.0853,8.3584);\n    );out ;'

In [6]:
osmrequest = {'data': compactOverpassQLstring}
osmurl = 'https://overpass-api.de/api/interpreter'

# Ask the API
osm = requests.get(osmurl, params=osmrequest)

### Reformat the JSON to fit in a Pandas Dataframe

The JSON can't be directyl imported to a Pandas Dataframe:

Thanks to [unutbu from stackoverflow.com](http://stackoverflow.com/questions/24848416/expected-string-or-unicode-when-reading-json-with-pandas) for fiddling this out!

In [7]:
osmdata = osm.json()
# print out the json
osmdata

{'version': 0.6,
 'generator': 'Overpass API 0.7.55.1009 5e627b63',
 'osm3s': {'timestamp_osm_base': '2019-11-25T07:37:02Z',
  'copyright': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.'},
 'elements': [{'type': 'node',
   'id': 1158988930,
   'lat': 47.0526073,
   'lon': 8.3076544,
   'tags': {'amenity': 'fountain',
    'name': 'Fritschibrunnen',
    'name:ru': 'Фонтан Фритши',
    'wikidata': 'Q62599999'}},
  {'type': 'node',
   'id': 1495294235,
   'lat': 47.0518254,
   'lon': 8.3030219,
   'tags': {'amenity': 'fountain', 'wikidata': 'Q29785697'}}]}

In [8]:
osmelements = osmdata['elements']
for dct in osmelements:
    for key, val in dct['tags'].items():
        dct[key] = val
    del dct['tags']

### Now put everything to the Pandas Dataframe

In [9]:
osmdf = pd.DataFrame(osmelements)

### Look at the whole Dataframe

In [10]:
osmdf.head(5)

,type,id,lat,lon,amenity,name,name:ru,wikidata
0,node,1158988930,47.052607,8.307654,fountain,Fritschibrunnen,Фонтан Фритши,Q62599999
1,node,1495294235,47.051825,8.303022,fountain,NaN,NaN,Q29785697


In [11]:
osmdfShrt =  osmdf[['lat','lon','wikidata']]

In [12]:
osmdfShrt.tail(5)

,lat,lon,wikidata
0,47.052607,8.307654,Q62599999
1,47.051825,8.303022,Q29785697


## Export to CSV

In [14]:
osmCsvFileName = "osmFountainsWithWikidataReference_"+nameOfRegion+"_"+dt.now().strftime(dtFmt)+".csv"
osmdfShrt.to_csv(osmCsvFileName, index=False)



In [15]:
print("wrote '"+osmCsvFileName+"' with "+str(len(osmdf))+" lines")

wrote 'osmFountainsWithWikidataReference_Luzern_191125_074112.csv' with 2 lines
